# Chapter 20: Optimization Techniques

Once you have profiled your code and identified bottlenecks, this notebook covers the
techniques available to make Python code faster and more memory-efficient. The golden rule:
**profile first, optimize algorithmic complexity, then micro-optimize**.

## Topics Covered
- **`__slots__`**: Memory optimization for classes
- **Algorithmic improvements**: O(n) vs O(n^2) with measurements
- **`dis` module**: Bytecode disassembly
- **Code object attributes**: `co_varnames`, `co_consts`, etc.
- **Generators vs list comprehensions**: Memory trade-offs
- **Local vs global variable access**: `LOAD_FAST` vs `LOAD_GLOBAL`
- **`collections` and built-in optimizations**
- **The GIL**: Threading vs multiprocessing
- **`ctypes` overview**: Calling C from Python
- **Optimization guidelines**: A practical workflow

## `__slots__` for Memory Optimization

By default, Python stores instance attributes in a per-object `__dict__` dictionary. For
classes with many instances, this is wasteful. Defining `__slots__` replaces `__dict__` with
a fixed-size struct, reducing memory per instance significantly.

In [ ]:
import sys


class PointDict:
    """Regular class with __dict__."""
    def __init__(self, x: float, y: float, z: float) -> None:
        self.x = x
        self.y = y
        self.z = z


class PointSlots:
    """Slotted class - no __dict__."""
    __slots__ = ("x", "y", "z")

    def __init__(self, x: float, y: float, z: float) -> None:
        self.x = x
        self.y = y
        self.z = z


# Compare single-instance memory
pd = PointDict(1.0, 2.0, 3.0)
ps = PointSlots(1.0, 2.0, 3.0)

print("Single instance memory:")
print(f"  PointDict:  {sys.getsizeof(pd)} + __dict__: {sys.getsizeof(pd.__dict__)} bytes")
print(f"  PointSlots: {sys.getsizeof(ps)} bytes (no __dict__)")

# Measure memory for many instances
n = 100_000
dict_objects = [PointDict(1.0, 2.0, 3.0) for _ in range(n)]
slots_objects = [PointSlots(1.0, 2.0, 3.0) for _ in range(n)]

# Approximate total memory (instance + __dict__ for dict version)
dict_total = sum(sys.getsizeof(o) + sys.getsizeof(o.__dict__) for o in dict_objects[:100]) / 100
slots_total = sum(sys.getsizeof(o) for o in slots_objects[:100]) / 100

print(f"\nEstimated per-object cost ({n:,} instances):")
print(f"  PointDict:  ~{dict_total:.0f} bytes/obj = ~{dict_total * n / 1024 / 1024:.1f} MB total")
print(f"  PointSlots: ~{slots_total:.0f} bytes/obj = ~{slots_total * n / 1024 / 1024:.1f} MB total")
print(f"  Savings: ~{(1 - slots_total / dict_total) * 100:.0f}%")

# __slots__ also prevents adding arbitrary attributes
try:
    ps.w = 4.0  # type: ignore[attr-defined]
except AttributeError as e:
    print(f"\nCannot add attributes to slotted class: {e}")

## Algorithmic Improvements: O(n) vs O(n^2)

The single most impactful optimization is choosing a better algorithm. A constant-factor
micro-optimization cannot save code that has the wrong Big-O complexity.

In [ ]:
import time


def has_duplicates_quadratic(data: list[int]) -> bool:
    """O(n^2): Check every pair for duplicates."""
    n = len(data)
    for i in range(n):
        for j in range(i + 1, n):
            if data[i] == data[j]:
                return True
    return False


def has_duplicates_linear(data: list[int]) -> bool:
    """O(n): Use a set for constant-time lookup."""
    seen: set[int] = set()
    for item in data:
        if item in seen:
            return True
        seen.add(item)
    return False


def has_duplicates_pythonic(data: list[int]) -> bool:
    """O(n): Idiomatic Python -- compare len to set."""
    return len(data) != len(set(data))


# Benchmark with increasing sizes
print(f"{'Size':>8} {'O(n^2)':>12} {'O(n) set':>12} {'Pythonic':>12}")
print("-" * 48)

for size in [1_000, 5_000, 10_000, 20_000]:
    # No duplicates (worst case for quadratic)
    data = list(range(size))

    start = time.perf_counter()
    has_duplicates_quadratic(data)
    t_quad = time.perf_counter() - start

    start = time.perf_counter()
    has_duplicates_linear(data)
    t_linear = time.perf_counter() - start

    start = time.perf_counter()
    has_duplicates_pythonic(data)
    t_pythonic = time.perf_counter() - start

    print(f"{size:>8,} {t_quad:>11.4f}s {t_linear:>11.6f}s {t_pythonic:>11.6f}s")

print("\nNote how O(n^2) grows ~4x when n doubles, while O(n) grows ~2x.")

## The `dis` Module: Bytecode Disassembly

Python compiles source code to bytecode, which the CPython virtual machine then executes.
The `dis` module lets you inspect this bytecode. This is useful for understanding *why*
one approach is faster than another at the lowest level.

In [ ]:
import dis


def add_numbers(a: int, b: int) -> int:
    """Simple addition function."""
    return a + b


def add_with_temp(a: int, b: int) -> int:
    """Addition with a temporary variable."""
    result = a + b
    return result


print("Bytecode for add_numbers:")
dis.dis(add_numbers)

print("\nBytecode for add_with_temp:")
dis.dis(add_with_temp)

print("\nKey instructions:")
print("  LOAD_FAST    - Load a local variable (fast, indexed by position)")
print("  BINARY_OP    - Perform a binary operation (+, -, *, etc.)")
print("  STORE_FAST   - Store to a local variable")
print("  RETURN_VALUE - Return the top of the stack")

In [ ]:
import dis

# Compare: list comprehension vs manual loop
def with_comprehension() -> list[int]:
    return [x * 2 for x in range(10)]

def with_loop() -> list[int]:
    result: list[int] = []
    for x in range(10):
        result.append(x * 2)
    return result

print("List comprehension bytecode:")
dis.dis(with_comprehension)

print("\n" + "=" * 60)
print("\nManual loop bytecode:")
dis.dis(with_loop)

print("\nThe comprehension uses a specialized BUILD_LIST + LIST_APPEND")
print("path that avoids the overhead of LOAD_ATTR for .append().")

## Code Object Attributes

Every function has a `__code__` object containing its compiled bytecode and metadata.
Inspecting these attributes reveals how Python organizes function internals.

In [ ]:
def example_function(x: int, y: int, z: int = 10) -> int:
    """A function with locals, constants, and a default."""
    multiplier = 2
    offset = 100
    result = (x + y + z) * multiplier + offset
    return result


code = example_function.__code__

print("Code object attributes:")
print(f"  co_argcount:   {code.co_argcount}")       # Number of positional args
print(f"  co_varnames:   {code.co_varnames}")       # Local variable names (args first)
print(f"  co_consts:     {code.co_consts}")         # Constant values used
print(f"  co_names:      {code.co_names}")          # Names used (global lookups)
print(f"  co_nlocals:    {code.co_nlocals}")        # Number of local variables
print(f"  co_stacksize:  {code.co_stacksize}")      # Max stack depth needed
print(f"  co_filename:   {code.co_filename}")       # Source file

# Constants are pre-computed at compile time
print(f"\nConstants include: docstring, default values, and literal numbers")
print(f"Note: co_varnames includes args first, then locals")

## Generator Expressions vs List Comprehensions: Memory

A list comprehension builds the entire list in memory. A generator expression yields
one item at a time, using near-constant memory regardless of data size. When you only
need to iterate once, generators are the better choice.

In [ ]:
import sys

# List comprehension: builds entire list in memory
list_comp = [x * x for x in range(1_000_000)]

# Generator expression: lazy, constant memory
gen_expr = (x * x for x in range(1_000_000))

print("Memory comparison (1M items):")
print(f"  List comprehension: {sys.getsizeof(list_comp):>12,} bytes")
print(f"  Generator expr:     {sys.getsizeof(gen_expr):>12,} bytes")

# Both produce the same result when consumed
print(f"\n  sum(list_comp): {sum(list_comp)}")
print(f"  sum(gen_expr):  {sum(gen_expr)}")

# Functions like sum(), min(), max(), any(), all() accept iterables
# so you can feed them generators directly
import timeit

t_list = timeit.timeit("sum([x*x for x in range(10_000)])", number=1000)
t_gen = timeit.timeit("sum(x*x for x in range(10_000))", number=1000)

print(f"\nsum() with 10k items, 1000 runs:")
print(f"  List comp: {t_list:.4f}s")
print(f"  Generator: {t_gen:.4f}s")
print("  Generator avoids allocating and freeing a temporary list.")

## Local Variable Access vs Global: LOAD_FAST vs LOAD_GLOBAL

Local variables are stored in a fixed-size array and accessed by index (`LOAD_FAST`).
Global and built-in lookups require dictionary lookups (`LOAD_GLOBAL`). In tight loops,
this difference matters.

In [ ]:
import dis
import timeit

GLOBAL_MULTIPLIER: int = 3


def use_global(data: list[int]) -> list[int]:
    """Access a global variable in a loop."""
    return [x * GLOBAL_MULTIPLIER for x in data]


def use_local(data: list[int]) -> list[int]:
    """Cache the global as a local variable."""
    multiplier = GLOBAL_MULTIPLIER  # One LOAD_GLOBAL, then LOAD_FAST
    return [x * multiplier for x in data]


# Show the bytecode difference
print("use_global bytecode:")
dis.dis(use_global)

print("\nuse_local bytecode:")
dis.dis(use_local)

# Benchmark
data = list(range(100_000))
t_global = timeit.timeit(lambda: use_global(data), number=100)
t_local = timeit.timeit(lambda: use_local(data), number=100)

print(f"\n100k items, 100 runs:")
print(f"  Global lookup:     {t_global:.4f}s")
print(f"  Local (cached):    {t_local:.4f}s")

## Collections and Built-in Optimizations

Python's standard library is written in C and heavily optimized. Using built-in functions
and `collections` types is almost always faster than reimplementing their logic in pure Python.

In [ ]:
import timeit
from collections import Counter, defaultdict, deque

# Built-in sum() vs manual loop
setup = "data = list(range(10_000))"

t_builtin = timeit.timeit("sum(data)", setup=setup, number=10_000)
t_manual = timeit.timeit(
    "total = 0\nfor x in data:\n    total += x",
    setup=setup,
    number=10_000,
)

print("Summing 10k ints, 10k runs:")
print(f"  Built-in sum():  {t_builtin:.4f}s")
print(f"  Manual loop:     {t_manual:.4f}s")
print(f"  Built-in is {t_manual / t_builtin:.1f}x faster")

# Counter vs manual counting
words = ["apple", "banana", "cherry"] * 3000

def manual_count(words: list[str]) -> dict[str, int]:
    counts: dict[str, int] = {}
    for w in words:
        counts[w] = counts.get(w, 0) + 1
    return counts

t_counter = timeit.timeit(lambda: Counter(words), number=1_000)
t_manual_c = timeit.timeit(lambda: manual_count(words), number=1_000)

print(f"\nCounting 9k words, 1k runs:")
print(f"  Counter():      {t_counter:.4f}s")
print(f"  Manual dict:    {t_manual_c:.4f}s")

# deque.appendleft() vs list.insert(0, x)
t_deque = timeit.timeit(
    "d.appendleft(1)",
    setup="from collections import deque; d = deque(range(10000))",
    number=100_000,
)
t_list_ins = timeit.timeit(
    "lst.insert(0, 1)",
    setup="lst = list(range(10000))",
    number=100_000,
)

print(f"\nPrepend 100k times to a 10k-item container:")
print(f"  deque.appendleft(): {t_deque:.4f}s")
print(f"  list.insert(0):     {t_list_ins:.4f}s")

## The GIL: Global Interpreter Lock

CPython has a **Global Interpreter Lock (GIL)** that allows only one thread to execute
Python bytecode at a time. This means:

- **Threading** is useful for I/O-bound work (network, disk) where threads spend most
  time waiting, but **not** for CPU-bound work.
- **Multiprocessing** spawns separate processes, each with its own GIL, enabling true
  parallelism for CPU-bound tasks.

> **Note**: Python 3.13 introduced an experimental free-threaded build (PEP 703) that
> can disable the GIL. This is still experimental and not the default.

In [ ]:
import time
import threading
import multiprocessing


def cpu_bound_work(n: int) -> int:
    """CPU-intensive computation."""
    total = 0
    for i in range(n):
        total += i * i
    return total


WORK_SIZE = 5_000_000
NUM_TASKS = 4

# Sequential execution
start = time.perf_counter()
for _ in range(NUM_TASKS):
    cpu_bound_work(WORK_SIZE)
t_sequential = time.perf_counter() - start

# Threaded execution (GIL limits parallelism for CPU work)
start = time.perf_counter()
threads = [threading.Thread(target=cpu_bound_work, args=(WORK_SIZE,)) for _ in range(NUM_TASKS)]
for t in threads:
    t.start()
for t in threads:
    t.join()
t_threaded = time.perf_counter() - start

# Multiprocessing execution (true parallelism)
start = time.perf_counter()
with multiprocessing.Pool(NUM_TASKS) as pool:
    pool.map(cpu_bound_work, [WORK_SIZE] * NUM_TASKS)
t_multiproc = time.perf_counter() - start

print(f"CPU-bound work: {NUM_TASKS} tasks x {WORK_SIZE:,} iterations")
print(f"  Sequential:      {t_sequential:.3f}s")
print(f"  Threading:       {t_threaded:.3f}s  (GIL prevents speedup)")
print(f"  Multiprocessing: {t_multiproc:.3f}s  (true parallelism)")
print(f"\nThreading speedup:       {t_sequential / t_threaded:.2f}x")
print(f"Multiprocessing speedup: {t_sequential / t_multiproc:.2f}x")

## ctypes Overview: Calling C Functions from Python

`ctypes` allows Python to call functions in shared C libraries (`.so` on Linux, `.dylib`
on macOS, `.dll` on Windows) without writing any C extension code. This is useful for
performance-critical sections or interfacing with existing C libraries.

This is a **conceptual overview** -- we will call functions from the C standard library
to demonstrate the mechanism.

In [ ]:
import ctypes
import ctypes.util
import sys

# Load the C standard library
if sys.platform == "darwin":
    libc = ctypes.CDLL("libSystem.B.dylib")
elif sys.platform == "linux":
    libc_path = ctypes.util.find_library("c")
    libc = ctypes.CDLL(libc_path)
else:
    libc = ctypes.CDLL("msvcrt")  # Windows

# Call C's abs() function
libc.abs.restype = ctypes.c_int
libc.abs.argtypes = [ctypes.c_int]

result = libc.abs(-42)
print(f"C abs(-42) = {result}")

# Call C's strlen() on a byte string
libc.strlen.restype = ctypes.c_size_t
libc.strlen.argtypes = [ctypes.c_char_p]

length = libc.strlen(b"Hello, ctypes!")
print(f"C strlen('Hello, ctypes!') = {length}")

# Conceptual: for real performance gains, you would:
# 1. Write a C function that performs your hot loop
# 2. Compile it into a shared library
# 3. Load it with ctypes and call it from Python
print("\nctypes workflow:")
print("  1. Write C code -> compile to .so / .dylib / .dll")
print("  2. ctypes.CDLL('mylib.so')")
print("  3. Set .restype and .argtypes for type safety")
print("  4. Call functions directly from Python")
print("\nAlternatives: cffi, Cython, pybind11, or the new C API")

## Optimization Guidelines: A Practical Workflow

Follow this order when optimizing Python code:

1. **Write correct code first** -- premature optimization is the root of all evil
2. **Profile** with `cProfile` to find actual bottlenecks (do not guess)
3. **Fix algorithmic complexity** -- O(n) vs O(n^2) matters far more than micro-tuning
4. **Use built-in functions and C-implemented types** (`sum`, `sorted`, `Counter`, `deque`)
5. **Reduce memory pressure** -- `__slots__`, generators, avoid unnecessary copies
6. **Micro-optimize hot loops** -- local variable caching, list comprehensions
7. **Consider C extensions** -- `ctypes`, `Cython`, `pybind11` for truly hot code
8. **Use multiprocessing** for CPU-bound parallelism (not threading)

In [ ]:
import timeit

# Putting it all together: optimizing a word frequency counter
# Step by step, from naive to optimized

sample_text = "the quick brown fox jumps over the lazy dog " * 10_000


def v1_naive(text: str) -> dict[str, int]:
    """Version 1: Naive implementation."""
    words = text.split()
    counts: dict[str, int] = {}
    for word in words:
        if word in counts:
            counts[word] = counts[word] + 1
        else:
            counts[word] = 1
    return counts


def v2_get_default(text: str) -> dict[str, int]:
    """Version 2: Use dict.get() with default."""
    words = text.split()
    counts: dict[str, int] = {}
    for word in words:
        counts[word] = counts.get(word, 0) + 1
    return counts


def v3_defaultdict(text: str) -> dict[str, int]:
    """Version 3: Use defaultdict(int)."""
    from collections import defaultdict
    words = text.split()
    counts: defaultdict[str, int] = defaultdict(int)
    for word in words:
        counts[word] += 1
    return dict(counts)


def v4_counter(text: str) -> dict[str, int]:
    """Version 4: Use Counter (C-optimized)."""
    from collections import Counter
    return dict(Counter(text.split()))


# Benchmark all versions
versions = [
    ("v1: naive if/else", v1_naive),
    ("v2: dict.get()", v2_get_default),
    ("v3: defaultdict", v3_defaultdict),
    ("v4: Counter", v4_counter),
]

print(f"Word frequency counting: {len(sample_text.split()):,} words")
print("=" * 45)

for name, func in versions:
    t = timeit.timeit(lambda f=func: f(sample_text), number=50)
    print(f"  {name:<22s} {t:.4f}s (50 runs)")

# Verify all produce the same result
results = [func(sample_text) for _, func in versions]
print(f"\nAll versions agree: {all(r == results[0] for r in results)}")

## Summary

### Optimization Hierarchy (Most to Least Impact)

| Priority | Technique | Typical Speedup |
|----------|-----------|----------------|
| 1 | **Algorithm choice** (O(n) vs O(n^2)) | 10x - 1000x+ |
| 2 | **Use C-implemented built-ins** (sum, sorted, Counter) | 2x - 10x |
| 3 | **Reduce memory** (__slots__, generators) | 30-50% memory savings |
| 4 | **Micro-optimize** (local vars, comprehensions) | 5-30% |
| 5 | **C extensions** (ctypes, Cython) | 10x - 100x for hot loops |
| 6 | **Parallelism** (multiprocessing for CPU-bound) | Up to Nx on N cores |

### Key Tools
- **`dis.dis()`**: See exactly what bytecode Python generates
- **`__code__` attributes**: Inspect function internals
- **`sys.getsizeof()`**: Measure memory of individual objects
- **`__slots__`**: Eliminate per-instance `__dict__` overhead
- **`ctypes`**: Call C functions without writing C extensions

### The Golden Rule
**"Make it work, make it right, make it fast -- in that order."**

Always profile before optimizing. The bottleneck is almost never where you think it is.